# 6. 전화번호, 계좌번호 경찰청 검색
- https://www.counterscam112.go.kr/phishing/searchPhone.do
- 해당 사이트를 크롤링하여 이전 단계에서 추출한 전화번호를 입력하여 검색 결과를 가져옴

## [1] 환경 설정 및 라이브러리 임포트

In [ ]:
# !pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, NoSuchElementException
import time

### 검색 타입 지정

In [11]:
# ✅ 검색할 전화번호
phone_number = "01058720546"  # 검색할 번호

# ✅ ChromeDriver 설정
options = Options()
# options.add_argument("--headless")  # 디버깅 시 주석처리
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
service = Service()  # chromedriver가 환경변수에 등록돼 있으면 이대로 OK

# ✅ 드라이버 실행
try:
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 10)

    # ✅ 사이트 접속
    driver.get("https://www.counterscam112.go.kr/phishing/searchPhone.do")

    # ✅ 입력창 대기 및 값 입력
    wait.until(EC.presence_of_element_located((By.ID, "tel_num")))
    input_box = driver.find_element(By.ID, "tel_num")
    
    print("📌 입력창 HTML:")
    print(input_box.get_attribute("outerHTML"))
    
    input_box.clear()
    input_box.send_keys(phone_number)
    
    # ✅ 검색 버튼 클릭
    search_btn = driver.find_element(By.CLASS_NAME, "ico_sch_btn")
    
    print("📌 검색 버튼 HTML:")
    print(search_btn.get_attribute("outerHTML"))
    
    search_btn.click()
    
    # ✅ 결과 로딩 대기
    wait.until(EC.presence_of_element_located((By.ID, "tel_num_result_data")))
    time.sleep(1)
    
    # ✅ 결과 추출 (요소 자체 및 텍스트 확인)
    total_elem = driver.find_element(By.ID, "search-totcnt")
    voice_elem = driver.find_element(By.ID, "search-voice-cnt")
    sms_elem = driver.find_element(By.ID, "search-sms-cnt")
    
    print("\n📌 전체 신고 건수 element:")
    print(total_elem.get_attribute("outerHTML"))
    print("➡ 텍스트:", total_elem.text.strip())
    
    print("\n📌 음성 신고 건수 element:")
    print(voice_elem.get_attribute("outerHTML"))
    print("➡ 텍스트:", voice_elem.text.strip())
    
    print("\n📌 문자 신고 건수 element:")
    print(sms_elem.get_attribute("outerHTML"))
    print("➡ 텍스트:", sms_elem.text.strip())
    
    # ✅ 숫자 변환
    total_count = int(total_elem.text.strip()) if total_elem.text.strip().isdigit() else 0
    voice_count = int(voice_elem.text.strip()) if voice_elem.text.strip().isdigit() else 0
    sms_count = int(sms_elem.text.strip()) if sms_elem.text.strip().isdigit() else 0
    
    # ✅ 최종 출력
    print(f"\n📊 총 신고 건수: {total_count}건")
    print(f"🔈 음성 신고 건수: {voice_count}건")
    print(f"💬 문자 신고 건수: {sms_count}건")

except WebDriverException as e:
    print("❌ WebDriver 예외 발생:", str(e))

except NoSuchElementException as e:
    print("❌ 요소를 찾을 수 없습니다:", str(e))

except Exception as e:
    print("❌ 예기치 못한 오류 발생:", str(e))

finally:
    try:
        driver.quit()
    except:
        pass

📌 입력창 HTML:
<input type="number" id="tel_num" oninput="javascript: if (this.value.length > this.maxLength) this.value = this.value.slice(0, this.maxLength);" placeholder="'-' 없이 숫자만 입력" title="전화번호를 -제외하고 입력(ex: 01012341234)" onkeyup="if(window.event.keyCode==13){}" name="voPh" maxlength="30">
📌 검색 버튼 HTML:
<button type="button" class="ico_sch_btn" onclick="fn_numb_sch();">검색</button>

📌 전체 신고 건수 element:
<span id="search-totcnt">3</span>
➡ 텍스트: 3

📌 음성 신고 건수 element:
<span class="type_count" id="search-voice-cnt">3</span>
➡ 텍스트: 3

📌 문자 신고 건수 element:
<span class="type_count" id="search-sms-cnt">0</span>
➡ 텍스트: 0

📊 총 신고 건수: 3건
🔈 음성 신고 건수: 3건
💬 문자 신고 건수: 0건


✅ 사용 예시 (다른 파일이나 Jupyter Notebook에서)

In [ ]:
from phone_checker import check_phone_number

result = check_phone_number("01012345678")
if result:
    print(f"📊 총 신고 건수: {result['total']}")
    print(f"🔈 음성 신고 건수: {result['voice']}")
    print(f"💬 문자 신고 건수: {result['sms']}")
else:
    print("⚠️ 조회 실패")